##  Initialization

In [ ]:
# Import packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
from itertools import permutations
import matplotlib.pyplot as plt
import copy
from IPython.display import display


## 1. Consumer-level flexibility models

1A) We first consider that this consumer has a single (fully) flexible load and a rooftop
PV panel. They want to utilize their flexible load and rooftop PV production to
minimize their daily energy procurement cost. Their maximum hourly PV production
is perfectly known, but can be fully curtailed. Their load is fully flexible in each hour,
however, the consumer wants to satisfy a minimum (total) energy consumption at the
end of the day. They can sell/buy electricity to/from the distribution grid at an hourly
electricity price, but must pay an hourly grid tariff fee for all net imports and exports.
We analyze how these cost structures can impact their flexibility and profits.

i) Formulate the optimization problem of this flexible consumer, describing its main
components (input data, decision variables, objective function, constraints) and
properties.



The optimization problem has the following:

Input data
- $p_t$: electricity price [DKK/kWh]  
- $g_t$: grid tariff [DKK/kWh]  
- $\bar{y}_t$: max PV generation [kWh]  
- $E_{\min}$: minimum daily energy consumption [kWh]  
- $c^{PV}$: marginal PV cost [DKK/kWh] (often 0)


ii) Formulate the dual problem, Lagrangian, and KKT conditions of this optimization
problem. Describe their properties.

iii) Without numerically solving this optimization problem, analyze its structure and
discuss qualitatively the characteristics of its optimal solutions. Which cost com-
ponents and constraints have the most impact on the solutions? Which constraint
are binding at optimality? How does the input data impact the optimal solutions?
In particular, compare the cases in which: the electricity prices and grid tariffs
are constant through the day or vary hourly. For which values of the cost compo-
nents, can you conclude that this optimization problem is equivalent to a simpler
optimization one? Tip: you can use the dual formulation and KKT conditions to
motivate your analysis

iv) Implement this optimization problem in a programming language of your choice.
Provide a well-documented working code.

v) Construct relevant scenarios of system-level cost structures based on the data pro-
vided, solve this optimization problem over these scenarios, analyze its optimal
solutions (primal and dual) and answer the following question: How do cost struc-
tures impact the consumers flexibility and profits? Present your insights in a
compact manner using adequate visual aids. Do these solutions align with your
analysis from Question 1.(a).iii.?